In [1]:
import sys

def config_paths(user, env_name):
    paths = ['',
             '/home/{0}/{1}/.env/bin'.format(user, env_name),
             '/usr/lib/python35.zip',
             '/usr/lib/python3.5',
             '/usr/lib/python3.5/plat-x86_64-linux-gnu',
             '/usr/lib/python3.5/lib-dynload',
             '/home/{0}/{1}/.env/lib/python3.5/site-packages'.format(user, env_name),
             '/home/{0}/{1}/.env/lib/python3.5/site-packages/IPython/extensions'.format(user, env_name),
             '/home/{0}/.ipython']

    for path in paths:
        sys.path.append(path)
        
config_paths('omri', 'my_gpu')

import os
import numpy as np
import cv2
import pickle
import matplotlib.pyplot as plt
import pandas as pd

from keras.models import Sequential
from keras import Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.utils import np_utils

from keras.models import model_from_json


TRAIN_DIR = '../'
# K = 20
# SAMPLE_NUM = 128 * K   
IMG_SIZE = 197

/home/omri/my_gpu/.env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load Model

In [2]:
json_file = open('my_vgg_all_1_epoch.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("my_vgg_all_1_epoch.h5")

# Read Test Data

In [3]:
X_test = np.zeros(shape = (12800, IMG_SIZE,IMG_SIZE,3))
indices = np.zeros(12800)
test_path = '../test'
files_list = os.listdir(test_path)
for im_ind, im_name in enumerate(files_list):
    im = cv2.imread(os.path.join(test_path, im_name))
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = cv2.resize(im, (IMG_SIZE, IMG_SIZE))
    im = im/255.0
    img_ind = int(im_name.split('.')[0])
    indices[im_ind] = img_ind
    X_test[im_ind] = im
    
missing_imgs = set(range(12800)) - set(indices)
for i, loc in enumerate(np.where(indices==0)[0]):
    indices[loc] = list(missing_imgs)[i]

# Predict and Save

In [4]:
predictions = loaded_model.predict(X_test)
y_test = np.argmax(predictions,axis=1)
filename = 'my_vgg_all_1_epoch.csv'
ans = pd.DataFrame({'id': [int(x) for x in indices],
                    'predicted': [int(x) for x in y_test + np.ones(len(y_test))]})

ans.sort_values(by='id').to_csv(filename, index=False)